# Segmentation Project
This is a project where YL is diving the country into different racial and income categories to identify people who are doing ministry in locations with the same economic and ethnic make up. All the census data pull in this project is divide the country up by Census Tracts according to those categories.

#### Racial Categories:
* White: White population (ages 10 - 24) > 55%
* Black or African American: Black or African American (ages 10 - 24) population > 55%
* Hispanic:  > Hispanic population (ages 10 - 24) > 55%
* Multicultural: No single racial category (ages 10 - 24) is > 55%
* Other: Another race other than White, Black, or Hispanic (ages 10 - 24) > 55% (Also, if there was 0 population (ages 10 - 24))

#### Income Categories:
* Median Household Income Fundable: Median Household Income > 50,000
* Median Household Income Not Fundable: Median Household Income < 50,000

#### From these two categories, 6 different segments were determined that they wanted to look at.
1. White Fundable
2. White Harder to Fund
3. Multicultural Fundable
4. Multicultural Harder to Fund
5. Black or African American Harder to Fund
6. Hispanic Harder to Fund

All other segments were classified as "Other"

In [1]:
## Import needed libraries ##
import pandas as pd
import requests
import json
import time
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 1000)
pd.options.display.max_seq_items = 2000
api_key = ENTER_API_KEY_HERE

#### Bring in all the needed datasets.
* Both data sets can be found on the MADS share point under Segmentation.
* variables was originally pulled from (https://www2.census.gov/data/api-documentation/2020-5yr-api-changes.csv). It is a complete list of all the fields in the ACS, their understandable name for the 2020 dataset, and if that name has changed from the 2019 ACS.
* States is just a list of the state numbers to include all 50 states plus Washington DC and exclude any other territories, etc.

In [2]:
variables = pd.read_csv('C:/Users/acoyne/Desktop/Census Tract Files/2020-5yr-api-changes.csv')


states = pd.read_excel('C:/Users/acoyne/Desktop/Census Tract Files/states_fips.xlsx')
states.STATE_FIPS = states.STATE_FIPS.astype(str)
# These lines of code are to correct any of the values that 0 in front that were automatically removed in excel. 
for num in range(len(states)):
    if len(states.iloc[num, 0]) < 2:
        states.iloc[num, 0] = "0" + states.iloc[num, 0]
states_list = list(states.STATE_FIPS)#.sort()
states_list.sort()

#### Create dictionaries for cleaning data.
* group_dict is just dict created of all the groups that need to be pulled from the api as well as their associated names which assist in naming the columns correctly for all the different races.
* cols_dict is created from the variables data set and is used to rename the columns to their understandable name as well as determine which columns should be dropped.

In [3]:
group_dict = {'B19013':'',
              'B01001A':'White',
              'B01001B':'Black Or African American',
              'B01001C':'American Indian And Alaska Native',
              'B01001D':'Asian Alone',
              'B01001E':'Native Hawaiian And Other Pacific Islander',
              'B01001F':'Some Other Race',
              'B01001G':'Two Or More Races',
              'B01001H':'White, Not Hispanic Or Latino',
              'B01001I':'Hispanic Or Latino'
             }

cols_dict = {}
for each in range(len(variables)):
    cols_dict[variables.iloc[each, 3]] = variables.iloc[each, 5]

In [4]:
def get_and_clean_data_set(group, state, ethnicity, location_type, year):
    '''
    This function will pull in data from the API based on group, state, location type, and year. It will then clean the data to 
    return a Data Frame that only has the columns that are listed in variables and also assigns the appropriate, readable, name 
    to each column as well as add the ethnicity to the column name (the census data has the same column names for all ethnicities).
    '''
    api_req = requests.get(f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,group({group})&for={location_type}:*&in=state:{state}&key={api_key}")
    data = pd.DataFrame(api_req.json())
    drop_list = []
    for each in range(len(data.iloc[0])):
        var_list = list(data.iloc[0])
        
        if (var_list[each] in cols_dict.keys()) or (var_list[each] in ('state', 'county', 'tract', 'GEO_ID')):
            try:
                data.iloc[0, each] = f"{ethnicity} " + cols_dict[var_list[each]].replace('Estimate!!','').replace('!!',', ')
            except:
                pass
        else:
            drop_list.append(each)
            
    data = data.drop(drop_list, axis = 1)
    data.columns = data.iloc[0]
    data = data.iloc[1:].reset_index(drop=True)
    
    return data

#### This code will gather all the income information for counties in the US. There are hundreds of tracts that have no income information. In those instances, the county level income will be used to calculate whether it is fundable or not.

In [5]:
census_county_df = pd.DataFrame()
beginning = time.time()
for state in states.STATE_FIPS:
    temp_df = get_and_clean_data_set(list(group_dict.keys())[0], state, group_dict[list(group_dict.keys())[0]], 'county', 2020)
    census_county_df = pd.concat([census_county_df, temp_df], axis=0)
end = time.time()
print('This took ' + str(end - beginning) + ' seconds.')

This took 41.764533281326294 seconds.


#### This next bit of code will pull census data at the tract level from all states and all groups and combine it into one big data set.

In [6]:
census_df = pd.DataFrame()
beginning = time.time()
for state in states.STATE_FIPS:
    temp_df = get_and_clean_data_set(list(group_dict.keys())[0], state, group_dict[list(group_dict.keys())[0]], 'tract', 2020)
    for each in range(len(group_dict))[1:]:
        temp_df = temp_df.merge(get_and_clean_data_set(list(group_dict.keys())[each], state, group_dict[list(group_dict.keys())[each]], 'tract', 2020).iloc[:, :-3], on = "GEO_ID", how = 'inner')
    
    census_df = pd.concat([census_df, temp_df], axis=0)
    
end = time.time()
print('This took ' + str(end - beginning) + ' seconds.')

This took 1204.9481692314148 seconds.


In [7]:
census_backup = census_df.copy() # create a back in case a mistake is made, that way you don't have to wait 20 minutes to pull data again

census_df = census_backup    # uncomment if you want to revert to the original data pull

In [8]:
census_df.head()

,Median household income in the past 12 months (in 2020 inflation-adjusted dollars),GEO_ID,state,county,tract,White Total:,"White Total:, Male:","White Total:, Male:, Under 5 years","White Total:, Male:, 5 to 9 years","White Total:, Male:, 10 to 14 years","White Total:, Male:, 15 to 17 years","White Total:, Male:, 18 and 19 years","White Total:, Male:, 20 to 24 years","White Total:, Male:, 25 to 29 years","White Total:, Male:, 30 to 34 years","White Total:, Male:, 35 to 44 years","White Total:, Male:, 45 to 54 years","White Total:, Male:, 55 to 64 years","White Total:, Male:, 65 to 74 years","White Total:, Male:, 75 to 84 years","White Total:, Male:, 85 years and over","White Total:, Female:","White Total:, Female:, Under 5 years","White Total:, Female:, 5 to 9 years","White Total:, Female:, 10 to 14 years","White Total:, Female:, 15 to 17 years","White Total:, Female:, 18 and 19 years","White Total:, Female:, 20 to 24 years","White Total:, Female:, 25 to 29 years","White Total:, Female:, 30 to 34 years","White Total:, Female:, 35 to 44 years","White Total:, Female:, 45 to 54 years","White Total:, Female:, 55 to 64 years","White Total:, Female:, 65 to 74 years","White Total:, Female:, 75 to 84 years","White Total:, Female:, 85 years and over",Black Or African American Total:,"Black Or African American Total:, Male:","Black Or African American Total:, Male:, Under 5 years","Black Or African American Total:, Male:, 5 to 9 years","Black Or African American Total:, Male:, 10 to 14 years","Black Or African American Total:, Male:, 15 to 17 years","Black Or African American Total:, Male:, 18 and 19 years","Black Or African American Total:, Male:, 20 to 24 years","Black Or African American Total:, Male:, 25 to 29 years","Black Or African American Total:, Male:, 30 to 34 years","Black Or African American Total:, Male:, 35 to 44 years","Black Or African American Total:, Male:, 45 to 54 years","Black Or African American Total:, Male:, 55 to 64 years","Black Or African American Total:, Male:, 65 to 74 years","Black Or African American Total:, Male:, 75 to 84 years","Black Or African American Total:, Male:, 85 years and over","Black Or African American Total:, Female:","Black Or African American Total:, Female:, Under 5 years","Black Or African American Total:, Female:, 5 to 9 years","Black Or African American Total:, Female:, 10 to 14 years","Black Or African American Total:, Female:, 15 to 17 years","Black Or African American Total:, Female:, 18 and 19 years","Black Or African American Total:, Female:, 20 to 24 years","Black Or African American Total:, Female:, 25 to 29 years","Black Or African American Total:, Female:, 30 to 34 years","Black Or African American Total:, Female:, 35 to 44 years","Black Or African American Total:, Female:, 45 to 54 years","Black Or African American Total:, Female:, 55 to 64 years","Black Or African American Total:, Female:, 65 to 74 years","Black Or African American Total:, Female:, 75 to 84 years","Black Or African American Total:, Female:, 85 years and over",American Indian And Alaska Native Total:,"American Indian And Alaska Native Total:, Male:","American Indian And Alaska Native Total:, Male:, Under 5 years","American Indian And Alaska Native Total:, Male:, 5 to 9 years","American Indian And Alaska Native Total:, Male:, 10 to 14 years","American Indian And Alaska Native Total:, Male:, 15 to 17 years","American Indian And Alaska Native Total:, Male:, 18 and 19 years","American Indian And Alaska Native Total:, Male:, 20 to 24 years","American Indian And Alaska Native Total:, Male:, 25 to 29 years","American Indian And Alaska Native Total:, Male:, 30 to 34 years","American Indian And Alaska Native Total:, Male:, 35 to 44 years","American Indian And Alaska Native Total:, Male:, 45 to 54 years","American Indian And Alaska Native Total:, Male:, 55 to 64 years","American Indian And Alaska Native Total:, Male:, 65 to 74 years","American Indian And Alaska Native Total:, Male:, 75 to 84 years","American Indian 

census_df = census_backup

In [8]:
census_df.shape # Check to make sure that the number of rows and columns is as expected.

(84414, 284)

In [12]:
census_county_df.head()

,county_fips,County Median Income
0,09003,76259
1,09005,81590
2,09009,71370
3,09011,75831
4,09015,67365


#### Now I want to merge the county level data onto it. Then that data can be used in the absences of the tract level data.
- first, the data has to be cleaned and a common column created for each that it can be merged on. In this instance, that is "county_fips"

In [10]:
census_county_df['county_fips'] = census_county_df['state'] + census_county_df['county']
census_county_df = census_county_df[['county_fips',' Median household income in the past 12 months (in 2020 inflation-adjusted dollars)']].rename(columns = {' Median household income in the past 12 months (in 2020 inflation-adjusted dollars)':'County Median Income'})
census_county_df['County Median Income'] = census_county_df['County Median Income'].astype(int)

In [11]:
census_df['county_fips'] = census_df['state'] + census_df['county']
census_df.iloc[:, 0] = census_df.iloc[:, 0].astype(int)
census_df = census_df.merge(census_county_df, on = 'county_fips', how = 'inner')
for num in range(len(census_df)):
    if census_df.iloc[num, 0] <= 0:
        census_df.iloc[num, 0] = census_df.iloc[num, -1]

#### Finally, export to a csv file.

census_df.to_csv('C:/Users/acoyne/Desktop/ct_2020.csv')

#### The population that is being used for our analysis is only for ages 10 - 24 so I want to get rid of all the population columns not related to that age range. Also, the White columns will not be used. To calculate the white population, the columns named White, Not Hispanic or Latino are used so that they are not double counted.

In [13]:
bad_columns = []
for item in census_df.columns:
    if '10 to 14 years' not in item and '15 to 17 years' not in item and '18 and 19 years' not in item and '20 to 24 years' not in item and item not in ('GEO_ID', 'state', 'county', 'tract', ' Median household income in the past 12 months (in 2020 inflation-adjusted dollars)'):
        bad_columns.append(item)

white_columns = ['White Total:, Male:, 10 to 14 years',
       'White Total:, Male:, 15 to 17 years',
       'White Total:, Male:, 18 and 19 years',
       'White Total:, Male:, 20 to 24 years',
       'White Total:, Female:, 10 to 14 years',
       'White Total:, Female:, 15 to 17 years',
       'White Total:, Female:, 18 and 19 years',
       'White Total:, Female:, 20 to 24 years',]

census_filter_df = census_df.drop(bad_columns + white_columns, axis = 1)

census_filter_df.iloc[:, 5:] = census_filter_df.iloc[:, 5:].astype(int)
census_filter_df.iloc[:, 0] = census_filter_df.iloc[:, 0].astype(int)

In [14]:
census_filter_df.head()

,Median household income in the past 12 months (in 2020 inflation-adjusted dollars),GEO_ID,state,county,tract,"Black Or African American Total:, Male:, 10 to 14 years","Black Or African American Total:, Male:, 15 to 17 years","Black Or African American Total:, Male:, 18 and 19 years","Black Or African American Total:, Male:, 20 to 24 years","Black Or African American Total:, Female:, 10 to 14 years","Black Or African American Total:, Female:, 15 to 17 years","Black Or African American Total:, Female:, 18 and 19 years","Black Or African American Total:, Female:, 20 to 24 years","American Indian And Alaska Native Total:, Male:, 10 to 14 years","American Indian And Alaska Native Total:, Male:, 15 to 17 years","American Indian And Alaska Native Total:, Male:, 18 and 19 years","American Indian And Alaska Native Total:, Male:, 20 to 24 years","American Indian And Alaska Native Total:, Female:, 10 to 14 years","American Indian And Alaska Native Total:, Female:, 15 to 17 years","American Indian And Alaska Native Total:, Female:, 18 and 19 years","American Indian And Alaska Native Total:, Female:, 20 to 24 years","Asian Alone Total:, Male:, 10 to 14 years","Asian Alone Total:, Male:, 15 to 17 years","Asian Alone Total:, Male:, 18 and 19 years","Asian Alone Total:, Male:, 20 to 24 years","Asian Alone Total:, Female:, 10 to 14 years","Asian Alone Total:, Female:, 15 to 17 years","Asian Alone Total:, Female:, 18 and 19 years","Asian Alone Total:, Female:, 20 to 24 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 10 to 14 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 15 to 17 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 18 and 19 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 20 to 24 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 10 to 14 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 15 to 17 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 18 and 19 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 20 to 24 years","Some Other Race Total:, Male:, 10 to 14 years","Some Other Race Total:, Male:, 15 to 17 years","Some Other Race Total:, Male:, 18 and 19 years","Some Other Race Total:, Male:, 20 to 24 years","Some Other Race Total:, Female:, 10 to 14 years","Some Other Race Total:, Female:, 15 to 17 years","Some Other Race Total:, Female:, 18 and 19 years","Some Other Race Total:, Female:, 20 to 24 years","Two Or More Races Total:, Male:, 10 to 14 years","Two Or More Races Total:, Male:, 15 to 17 years","Two Or More Races Total:, Male:, 18 and 19 years","Two Or More Races Total:, Male:, 20 to 24 years","Two Or More Races Total:, Female:, 10 to 14 years","Two Or More Races Total:, Female:, 15 to 17 years","Two Or More Races Total:, Female:, 18 and 19 years","Two Or More Races Total:, Female:, 20 to 24 years","White, Not Hispanic Or Latino Total:, Male:, 10 to 14 years","White, Not Hispanic Or Latino Total:, Male:, 15 to 17 years","White, Not Hispanic Or Latino Total:, Male:, 18 and 19 years","White, Not Hispanic Or Latino Total:, Male:, 20 to 24 years","White, Not Hispanic Or Latino Total:, Female:, 10 to 14 years","White, Not Hispanic Or Latino Total:, Female:, 15 to 17 years","White, Not Hispanic Or Latino Total:, Female:, 18 and 19 years","White, Not Hispanic Or Latino Total:, Female:, 20 to 24 years","Hispanic Or Latino Total:, Male:, 10 to 14 years","Hispanic Or Latino Total:, Male:, 15 to 17 years","Hispanic Or Latino Total:, Male:, 18 and 19 years","Hispanic Or Latino Total:, Male:, 20 to 24 years","Hispanic Or Latino Total:, Female:, 10 to 14 years","Hispanic Or Latino Total:, Female:, 15 to 17 years","Hispanic Or Latino Total:, Female:, 18 and 19 years","Hispanic Or Latino Total:, Female:, 20 to 24 years"
0,250001,1400000US09001010102,09,001,010102,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,30,7,27,8,27,176,69,107,76,325,45,27,80,4,0,0,0,

census_filter_df.to_csv('C:/Users/acoyne/Desktop/ct_2020_filtered.csv')

In [15]:
census_filter_df.iloc[0, 5:13].sum()

4

#### Now I need to new coloumns for the total population, the  total population for each racial category, and the percentage of the total population for each race for each row.

In [16]:
census_filter_df['Black Or African American'] = 0
census_filter_df['American Indian And Alaska Native'] = 0
census_filter_df['Asian Alone'] = 0
census_filter_df['Native Hawaiian And Other Pacific Islander'] = 0
census_filter_df['Some Other Race'] = 0
census_filter_df['Two Or More Races'] = 0
census_filter_df['White, Not Hispanic Or Latino'] = 0
census_filter_df['Hispanic Or Latino'] = 0
census_filter_df['Total'] = 0
census_filter_df['% Black Or African American'] = 0
census_filter_df['% American Indian And Alaska Native'] = 0
census_filter_df['% Asian Alone'] = 0
census_filter_df['% Native Hawaiian And Other Pacific Islander'] = 0
census_filter_df['% Some Other Race'] = 0
census_filter_df['% Two Or More Races'] = 0
census_filter_df['% White, Not Hispanic Or Latino'] = 0
census_filter_df['% Hispanic Or Latino'] = 0


for row in range(len(census_filter_df)):
    census_filter_df.iloc[row, -17] = census_filter_df.iloc[row, 5:13].sum()
    census_filter_df.iloc[row, -16] = census_filter_df.iloc[row, 13:21].sum()
    census_filter_df.iloc[row, -15] = census_filter_df.iloc[row, 21:29].sum()
    census_filter_df.iloc[row, -14] = census_filter_df.iloc[row, 29:37].sum()
    census_filter_df.iloc[row, -13] = census_filter_df.iloc[row, 37:45].sum()
    census_filter_df.iloc[row, -12] = census_filter_df.iloc[row, 45:53].sum()
    census_filter_df.iloc[row, -11] = census_filter_df.iloc[row, 53:61].sum()
    census_filter_df.iloc[row, -10] = census_filter_df.iloc[row, 61:69].sum()
    
for row in range(len(census_filter_df)):
    census_filter_df.iloc[row, -9] = census_filter_df.iloc[row, -17:-9].sum()
    
for row in range(len(census_filter_df)):    
    if census_filter_df.iloc[row, -9] == 0:
        census_filter_df.iloc[row, -8] = 0
        census_filter_df.iloc[row, -7] = 0
        census_filter_df.iloc[row, -6] = 0
        census_filter_df.iloc[row, -5] = 0
        census_filter_df.iloc[row, -4] = 0
        census_filter_df.iloc[row, -3] = 0
        census_filter_df.iloc[row, -2] = 0
        census_filter_df.iloc[row, -1] = 0
    else:
        census_filter_df.iloc[row, -8] = census_filter_df.iloc[row, -17]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -7] = census_filter_df.iloc[row, -16]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -6] = census_filter_df.iloc[row, -15]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -5] = census_filter_df.iloc[row, -14]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -4] = census_filter_df.iloc[row, -13]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -3] = census_filter_df.iloc[row, -12]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -2] = census_filter_df.iloc[row, -11]/census_filter_df.iloc[row, -9]
        census_filter_df.iloc[row, -1] = census_filter_df.iloc[row, -10]/census_filter_df.iloc[row, -9]
        
census_filter_df['race_ethincity_index'] = ''
census_filter_df['median_household_income_index'] = ''

for each in range(len(census_filter_df)):
    if census_filter_df.iloc[each, -11] == 0:
        census_filter_df.iloc[each, -2] = 'Other'
    if census_filter_df.iloc[each, -10] > .55:
        census_filter_df.iloc[each, -2] = 'Black or African American'
    elif census_filter_df.iloc[each, -3] > .55:
        census_filter_df.iloc[each, -2] = 'Hispanic or Latino/a'
    elif census_filter_df.iloc[each, -4] > .55:
        census_filter_df.iloc[each, -2] = 'White'
    elif census_filter_df.iloc[each, -9] < .55 and census_filter_df.iloc[each, -8] < .55 and census_filter_df.iloc[each, -7] < .55 and census_filter_df.iloc[each, -6] < .55 and census_filter_df.iloc[each, -5] < .55:
        census_filter_df.iloc[each, -2] = 'Multicultural'
    else:
        census_filter_df.iloc[each, -2] = 'Other'
        
    if census_filter_df.iloc[each, 0] < 50000:
        census_filter_df.iloc[each, -1] = 'Median Household Income Not Fundable'
    else:
        census_filter_df.iloc[each, -1] = 'Median Household Income Fundable'

census_filter_df['YL_Segment'] = 'Blank'
for each in range(len(census_filter_df)):
    if census_filter_df.iloc[each, -3] == 'White' and census_filter_df.iloc[each, -2] == 'Median Household Income Fundable':
        census_filter_df.iloc[each, -1] = 'White Fundable'
    elif census_filter_df.iloc[each, -3] == 'White' and census_filter_df.iloc[each, -2] == 'Median Household Income Not Fundable':
        census_filter_df.iloc[each, -1] = 'White Harder to Fund'
    elif census_filter_df.iloc[each, -3] == 'Multicultural' and census_filter_df.iloc[each, -2] == 'Median Household Income Fundable':
        census_filter_df.iloc[each, -1] = 'Multicultural Fundable'
    elif census_filter_df.iloc[each, -3] == 'Multicultural' and census_filter_df.iloc[each, -2] == 'Median Household Income Not Fundable':
        census_filter_df.iloc[each, -1] = 'Multicultural Harder to Fund'
    elif census_filter_df.iloc[each, -3] == 'Black or African American' and census_filter_df.iloc[each, -2] == 'Median Household Income Not Fundable':
        census_filter_df.iloc[each, -1] = 'Black or African American Harder to Fund'
    elif census_filter_df.iloc[each, -3] == 'Hispanic or Latino/a' and census_filter_df.iloc[each, -2] == 'Median Household Income Not Fundable':
        census_filter_df.iloc[each, -1] = 'Hispanic or Latino/a Harder to Fund'
    else:
        census_filter_df.iloc[each, -1] = 'Other'

#### Finally, just need to rearrange a few columns.

In [19]:
census_filter_df = census_filter_df.iloc[:, [1,2,3,4,0] + list(range(5, len(census_filter_df.columns)))]

In [20]:
census_filter_df = census_filter_df.sort_values('GEO_ID').reset_index(drop=True)

In [21]:
census_filter_df.head()

,GEO_ID,state,county,tract,Median household income in the past 12 months (in 2020 inflation-adjusted dollars),"Black Or African American Total:, Male:, 10 to 14 years","Black Or African American Total:, Male:, 15 to 17 years","Black Or African American Total:, Male:, 18 and 19 years","Black Or African American Total:, Male:, 20 to 24 years","Black Or African American Total:, Female:, 10 to 14 years","Black Or African American Total:, Female:, 15 to 17 years","Black Or African American Total:, Female:, 18 and 19 years","Black Or African American Total:, Female:, 20 to 24 years","American Indian And Alaska Native Total:, Male:, 10 to 14 years","American Indian And Alaska Native Total:, Male:, 15 to 17 years","American Indian And Alaska Native Total:, Male:, 18 and 19 years","American Indian And Alaska Native Total:, Male:, 20 to 24 years","American Indian And Alaska Native Total:, Female:, 10 to 14 years","American Indian And Alaska Native Total:, Female:, 15 to 17 years","American Indian And Alaska Native Total:, Female:, 18 and 19 years","American Indian And Alaska Native Total:, Female:, 20 to 24 years","Asian Alone Total:, Male:, 10 to 14 years","Asian Alone Total:, Male:, 15 to 17 years","Asian Alone Total:, Male:, 18 and 19 years","Asian Alone Total:, Male:, 20 to 24 years","Asian Alone Total:, Female:, 10 to 14 years","Asian Alone Total:, Female:, 15 to 17 years","Asian Alone Total:, Female:, 18 and 19 years","Asian Alone Total:, Female:, 20 to 24 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 10 to 14 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 15 to 17 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 18 and 19 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 20 to 24 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 10 to 14 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 15 to 17 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 18 and 19 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 20 to 24 years","Some Other Race Total:, Male:, 10 to 14 years","Some Other Race Total:, Male:, 15 to 17 years","Some Other Race Total:, Male:, 18 and 19 years","Some Other Race Total:, Male:, 20 to 24 years","Some Other Race Total:, Female:, 10 to 14 years","Some Other Race Total:, Female:, 15 to 17 years","Some Other Race Total:, Female:, 18 and 19 years","Some Other Race Total:, Female:, 20 to 24 years","Two Or More Races Total:, Male:, 10 to 14 years","Two Or More Races Total:, Male:, 15 to 17 years","Two Or More Races Total:, Male:, 18 and 19 years","Two Or More Races Total:, Male:, 20 to 24 years","Two Or More Races Total:, Female:, 10 to 14 years","Two Or More Races Total:, Female:, 15 to 17 years","Two Or More Races Total:, Female:, 18 and 19 years","Two Or More Races Total:, Female:, 20 to 24 years","White, Not Hispanic Or Latino Total:, Male:, 10 to 14 years","White, Not Hispanic Or Latino Total:, Male:, 15 to 17 years","White, Not Hispanic Or Latino Total:, Male:, 18 and 19 years","White, Not Hispanic Or Latino Total:, Male:, 20 to 24 years","White, Not Hispanic Or Latino Total:, Female:, 10 to 14 years","White, Not Hispanic Or Latino Total:, Female:, 15 to 17 years","White, Not Hispanic Or Latino Total:, Female:, 18 and 19 years","White, Not Hispanic Or Latino Total:, Female:, 20 to 24 years","Hispanic Or Latino Total:, Male:, 10 to 14 years","Hispanic Or Latino Total:, Male:, 15 to 17 years","Hispanic Or Latino Total:, Male:, 18 and 19 years","Hispanic Or Latino Total:, Male:, 20 to 24 years","Hispanic Or Latino Total:, Female:, 10 to 14 years","Hispanic Or Latino Total:, Female:, 15 to 17 years","Hispanic Or Latino Total:, Female:, 18 and 19 years","Hispanic Or Latino Total:, Female:, 20 to 24 years",Black Or African American,American Indian And Alaska Native,Asian Alone,Native Hawaiian And Other Pacific Islander,Some Other Race,Two Or More Races,"White, Not Hispanic Or Latin

In [130]:
census_filter_df.to_csv('C:/Users/acoyne/Desktop/census_transformed_2020_final_w_county_incomes.csv')

#### The following steps are optional if you want attach this census data to the census tract geometry and create a shape file from it to put on a map.

In [22]:
new_df = census_filter_df.copy()

In [23]:
new_df['GEOID'] = new_df['GEO_ID'][-11:]
for each in range(len(new_df)):
    new_df.iloc[each, -1] = new_df.iloc[each, 0][-11:]

In [24]:
new_df.head()

,GEO_ID,state,county,tract,Median household income in the past 12 months (in 2020 inflation-adjusted dollars),"Black Or African American Total:, Male:, 10 to 14 years","Black Or African American Total:, Male:, 15 to 17 years","Black Or African American Total:, Male:, 18 and 19 years","Black Or African American Total:, Male:, 20 to 24 years","Black Or African American Total:, Female:, 10 to 14 years","Black Or African American Total:, Female:, 15 to 17 years","Black Or African American Total:, Female:, 18 and 19 years","Black Or African American Total:, Female:, 20 to 24 years","American Indian And Alaska Native Total:, Male:, 10 to 14 years","American Indian And Alaska Native Total:, Male:, 15 to 17 years","American Indian And Alaska Native Total:, Male:, 18 and 19 years","American Indian And Alaska Native Total:, Male:, 20 to 24 years","American Indian And Alaska Native Total:, Female:, 10 to 14 years","American Indian And Alaska Native Total:, Female:, 15 to 17 years","American Indian And Alaska Native Total:, Female:, 18 and 19 years","American Indian And Alaska Native Total:, Female:, 20 to 24 years","Asian Alone Total:, Male:, 10 to 14 years","Asian Alone Total:, Male:, 15 to 17 years","Asian Alone Total:, Male:, 18 and 19 years","Asian Alone Total:, Male:, 20 to 24 years","Asian Alone Total:, Female:, 10 to 14 years","Asian Alone Total:, Female:, 15 to 17 years","Asian Alone Total:, Female:, 18 and 19 years","Asian Alone Total:, Female:, 20 to 24 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 10 to 14 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 15 to 17 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 18 and 19 years","Native Hawaiian And Other Pacific Islander Total:, Male:, 20 to 24 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 10 to 14 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 15 to 17 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 18 and 19 years","Native Hawaiian And Other Pacific Islander Total:, Female:, 20 to 24 years","Some Other Race Total:, Male:, 10 to 14 years","Some Other Race Total:, Male:, 15 to 17 years","Some Other Race Total:, Male:, 18 and 19 years","Some Other Race Total:, Male:, 20 to 24 years","Some Other Race Total:, Female:, 10 to 14 years","Some Other Race Total:, Female:, 15 to 17 years","Some Other Race Total:, Female:, 18 and 19 years","Some Other Race Total:, Female:, 20 to 24 years","Two Or More Races Total:, Male:, 10 to 14 years","Two Or More Races Total:, Male:, 15 to 17 years","Two Or More Races Total:, Male:, 18 and 19 years","Two Or More Races Total:, Male:, 20 to 24 years","Two Or More Races Total:, Female:, 10 to 14 years","Two Or More Races Total:, Female:, 15 to 17 years","Two Or More Races Total:, Female:, 18 and 19 years","Two Or More Races Total:, Female:, 20 to 24 years","White, Not Hispanic Or Latino Total:, Male:, 10 to 14 years","White, Not Hispanic Or Latino Total:, Male:, 15 to 17 years","White, Not Hispanic Or Latino Total:, Male:, 18 and 19 years","White, Not Hispanic Or Latino Total:, Male:, 20 to 24 years","White, Not Hispanic Or Latino Total:, Female:, 10 to 14 years","White, Not Hispanic Or Latino Total:, Female:, 15 to 17 years","White, Not Hispanic Or Latino Total:, Female:, 18 and 19 years","White, Not Hispanic Or Latino Total:, Female:, 20 to 24 years","Hispanic Or Latino Total:, Male:, 10 to 14 years","Hispanic Or Latino Total:, Male:, 15 to 17 years","Hispanic Or Latino Total:, Male:, 18 and 19 years","Hispanic Or Latino Total:, Male:, 20 to 24 years","Hispanic Or Latino Total:, Female:, 10 to 14 years","Hispanic Or Latino Total:, Female:, 15 to 17 years","Hispanic Or Latino Total:, Female:, 18 and 19 years","Hispanic Or Latino Total:, Female:, 20 to 24 years",Black Or African American,American Indian And Alaska Native,Asian Alone,Native Hawaiian And Other Pacific Islander,Some Other Race,Two Or More Races,"White, Not Hispanic Or Latin

In [25]:
import geopandas as gpd

geoms = gpd.read_file('C:/Users/acoyne/census_tract_geojson/all_tracts/Shape File/all_usa_ct.shp')
geoms = geoms[['GEOID','geometry']]
ct_geoms_comb = geoms.merge(new_df, on = 'GEOID', how = 'inner')

In [26]:
len(geoms)

84414

In [27]:
len(new_df)

84414

In [28]:
len(ct_geoms_comb)

84414

In [29]:
type(ct_geoms_comb)

geopandas.geodataframe.GeoDataFrame

#### I only want these particular 3 columns because I am making the Map in Power BI but need these columns to create a feature layer in ArcGIS Online that will be used in Power BI. Uncomment if you want to do the same.

ct_geoms_comb = ct_geoms_comb[['GEOID', 'YL_Segment', 'geometry']]

In [34]:
ct_geoms_comb.to_file('C:/Users/acoyne/census_tract_geojson/all_tracts/Shape File/ct_2020_shape_file_just_shapes.shp')

C:\Users\acoyne\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
